<a href="https://colab.research.google.com/github/kachytronico/colab-PIA/blob/main/206_La_maldici%C3%B3n_de_la_dimensionalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# La maldición de la dimensionalidad

Nuestro sexto cuadernillo (con un título muy novelístico, como puedes ver) nos servirá para explicar el problema de trabajar con datos con muchas dimensiones.

Realmente, ya hemos visto este problema con las _iris-setosa_, ¿recuerdas que tuvimos que comprimir algunas _features_ porque no podíamos representar todo el dataset en 2D por tener 4 dimensiones?

Y es que este problema es uno de los más complicados de atacar. ¿Qué información es la más importante? Tus clientes no van a ver las tablas tan bien como verán los gráficos, así que es especialmente importante que puedas reducir lo máximo posible las dimensiones de tus conjuntos de datos.

Y no solo para los humanos, los modelos también tendrán que trabajar con información que, si tiene muchas dimensiones, será mucho más difícil de tratar.

En general, en este cuadernillo veremos técnicas para tratar la [**MALDICIÓN DE LA DIMENSIONALIDAD**](https://en.wikipedia.org/wiki/Curse_of_dimensionality) _(no eres el Dr. Strange, ten cuidado ;)_.

## Explicabilidad y selección de las variables más importantes

Durante los cuadernillos anteriores hemos seleccionado a mano todas las columnas que nos interesaban: hemos eliminado las inútiles y hemos codificado aquellas columnas categóricas.

Ahora es el momento de seleccionar las variables más importantes de las que nos quedan. Aunque para nosotros todas sean importantes, para un modelo pueden no serlo todas por igual (y de hecho, no lo serán). Todas estas técnicas se denominan **técnicas de reducción de la dimensionalidad**.

Es importante observar que estas técnicas, aunque son de preprocesamiento, dependen del propio modelo (modelos distintos pueden fijarse en variables distintas), por lo que tendremos que usar y entrenar modelos aquí y ahora (pasaré de _pies puntillas_ por estos conceptos, dado que nos centraremos en los modelos a partir del Tema 3).

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

Voy a importar el selector de características y un modelo del grupo de modelos supervisados llamado KNN (ya lo hemos visto antes, en su versión "imputador"). Este modelo asigna sus predicciones dependiendo de los casos más similares que haya visto.

En este caso, mi objetivo será predecir la clase de los viajeros del titanic dependiendo de sus demás características.

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Análisis Exploratorio de Datos

Recuerda que el primer paso ante un nuevo problema es el **AED**, puedes verlo en ```103```.

## Preprocesamiento de la información

Vamos a procesar todo el conjunto de datos. Esto lo hemos visto a medias en los cuadernos de esta unidad. Lo vamos a resumir aquí:

### Columnas inútiles y valores sin sentido

Elimino todas las columnas inútiles, codificadas (quiero hacer mi propia codificación) y que no tengan sentido.

Inútiles:
- sibsp, parch, who, adult_male, deck (visto anteriormente).

Codificadas:
- survived, pclass, embarked (visto arriba).

Sin sentido:
- alive (me da igual si sobrevivió o no, quiero saber su clase).

In [ ]:
df = df.drop(columns=["survived", "pclass", "sibsp", "parch", "embarked", "who", "adult_male", "deck", "alive"])
df.head()

,sex,age,fare,class,embark_town,alone
0,male,22.0,7.2500,Third,Southampton,False
1,female,38.0,71.2833,First,Cherbourg,False
2,female,26.0,7.9250,Third,Southampton,True
3,female,35.0,53.1000,First,Southampton,False
4,male,35.0,8.0500,Third,Southampton,True


### Columnas categóricas

Vamos a codificar todas las columnas categóricas que tengamos aquí.

- ```sex```: dos valores, así que podemos usar 0 y 1.
- ```class```: tres valores, donde _first_ es mejor que _second_ y _second_ mejor que _third_.
- ```embark_town```: tres valores, sin orden. Codificamos usando un codificador binario.
- ```alone```: dos valores, así que usamos 0 y 1.

In [ ]:
df.sex = df.sex.apply(lambda v: 0 if v == "male" else 1 if v == "female" else v)
df.head()

,sex,age,fare,class,embark_town,alone
0,0,22.0,7.2500,Third,Southampton,False
1,1,38.0,71.2833,First,Cherbourg,False
2,1,26.0,7.9250,Third,Southampton,True
3,1,35.0,53.1000,First,Southampton,False
4,0,35.0,8.0500,Third,Southampton,True


In [ ]:
df.sex.unique() # todos han cambiado

array([0, 1])

In [ ]:
df["class"] = df["class"].apply(lambda v: 1 if v == "First" else 2 if v == "Second" else 3 if v == "Third" else v)
df.head()

,sex,age,fare,class,embark_town,alone
0,0,22.0,7.2500,3,Southampton,False
1,1,38.0,71.2833,1,Cherbourg,False
2,1,26.0,7.9250,3,Southampton,True
3,1,35.0,53.1000,1,Southampton,False
4,0,35.0,8.0500,3,Southampton,True


In [ ]:
df["class"].unique() # todos han cambiado

[3, 1, 2]
Categories (3, int64): [1, 2, 3]

In [ ]:
def binary_categorizer(dataframe, column, code_map: dict = None, cols: int = None):
  # resultados
  result = []

  # puede ser que me obliguen a que haya un número determinado de columnas
  if not cols:
    cols = math.ceil(math.log2(len(dataframe[column].unique()))) # aplico la fórmula de log_2_n y lo aproximo al número más grande

  # puede ser que no se conozca el mapa y tenga que inferirlo
  if not code_map:
    code_map = {value: key for key, value in enumerate(dataframe[column].unique())} # creo el mapa de forma genérica si no existe

  # realizo la codificación a binario, comprobando que puedo hacerlo para todos los valores
  for value in dataframe[column]:
    code = code_map[value] # recojo el código asignado
    b_code = format(code, "b") # lo convierto a binario

    if len(b_code) > cols: # cols no puede ser más pequeño que el código
      raise Exception(f"El número de columnas ({cols}) es demasiado pequeño para empaquetar la información ({len(b_code)}). Modifica el valor del atributo cols.")

    # realizo la codificación
    b_code_a = b_code.rjust(cols, "0") # lo formateo hasta tamaño cols rellenando con 0
    _value = list(b_code_a) # lo convierto a lista: cada elemento en una posición diferente 00 -> ["0", "0"]
    result.append(list(map(lambda v: int(v), _value))) # convierto la lista en una lista de enteros ["0", "0"] -> [0, 0]

  # defino las nuevas columnas y las añado a mi df
  new_columns_name = [f"{column}_{i}" for i in range(len(list(result[0])))] # les daré nombre a las nuevas columnas
  result_df = pd.DataFrame(result, index=dataframe.index, columns=new_columns_name) # creo un nuevo df con los resultados
  dataframe = pd.concat([dataframe, result_df], axis=1) # lo añado en el eje X respetando el orden
  return dataframe.drop(columns=[column]), code_map # también devuelvo el mapa de códigos, me será útil

In [ ]:
df, code_map = binary_categorizer(df, "embark_town")
df.head()

,sex,age,fare,class,alone,embark_town_0,embark_town_1
0,0,22.0,7.2500,3,False,0,0
1,1,38.0,71.2833,1,False,0,1
2,1,26.0,7.9250,3,True,0,0
3,1,35.0,53.1000,1,False,0,0
4,0,35.0,8.0500,3,True,0,0


In [ ]:
df.alone = df.alone.apply(lambda v: int(v)) # codificación de bool a 0 o 1
df.head()

,sex,age,fare,class,alone,embark_town_0,embark_town_1
0,0,22.0,7.2500,3,0,0,0
1,1,38.0,71.2833,1,0,0,1
2,1,26.0,7.9250,3,1,0,0
3,1,35.0,53.1000,1,0,0,0
4,0,35.0,8.0500,3,1,0,0


In [ ]:
df.alone.unique() # todos han cambiado

array([0, 1])

Ahora todo nuestro ```dataset``` contiene solo números.

In [ ]:
df

,sex,age,fare,class,alone,embark_town_0,embark_town_1
0,0,22.0,7.2500,3,0,0,0
1,1,38.0,71.2833,1,0,0,1
2,1,26.0,7.9250,3,1,0,0
3,1,35.0,53.1000,1,0,0,0
4,0,35.0,8.0500,3,1,0,0
...,...,...,...,...,...,...,...
886,0,27.0,13.0000,2,1,0,0
887,1,19.0,30.0000,1,1,0,0
888,1,NaN,23.4500,3,0,0,0
889,0,26.0,30.0000,1,1,0,1


### Datos para entrenar y etiquetas

Ahora, voy a separar los datos que usaré para entrenar mi modelo de la etiqueta que quiero predecir.

In [ ]:
X, y = df.drop(columns = "class"), df["class"]

Echemos un ojo:

In [ ]:
X

,sex,age,fare,alone,embark_town_0,embark_town_1
0,0,22.0,7.2500,0,0,0
1,1,38.0,71.2833,0,0,1
2,1,26.0,7.9250,1,0,0
3,1,35.0,53.1000,0,0,0
4,0,35.0,8.0500,1,0,0
...,...,...,...,...,...,...
886,0,27.0,13.0000,1,0,0
887,1,19.0,30.0000,1,0,0
888,1,NaN,23.4500,0,0,0
889,0,26.0,30.0000,1,0,1


In [ ]:
y

,class
0,3
1,1
2,3
3,1
4,3
...,...
886,2
887,1
888,3
889,1


### Valores nulos

En la columna ```age``` teníamos nulos, vamos a imputarlos.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
si = SimpleImputer()
si = si.fit(X.age.to_numpy().reshape(-1, 1))
pred = si.transform(X.age.to_numpy().reshape(-1, 1))
pred[10:15] # 10:15 es arbitrario, para mostrarte un ejemplo

array([[ 4.],
       [58.],
       [20.],
       [39.],
       [14.]])

In [ ]:
X.age = pred
X

,sex,age,fare,alone,embark_town_0,embark_town_1
0,0,22.000000,7.2500,0,0,0
1,1,38.000000,71.2833,0,0,1
2,1,26.000000,7.9250,1,0,0
3,1,35.000000,53.1000,0,0,0
4,0,35.000000,8.0500,1,0,0
...,...,...,...,...,...,...
886,0,27.000000,13.0000,1,0,0
887,1,19.000000,30.0000,1,0,0
888,1,29.699118,23.4500,0,0,0
889,0,26.000000,30.0000,1,0,1


In [ ]:
X.info() # sin nulos y todo números

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sex            891 non-null    int64  
 1   age            891 non-null    float64
 2   fare           891 non-null    float64
 3   alone          891 non-null    int64  
 4   embark_town_0  891 non-null    int64  
 5   embark_town_1  891 non-null    int64  
dtypes: float64(2), int64(4)
memory usage: 41.9 KB


Este conjunto de datos está preparado para ser utilizado.

## Reducción de la dimensionalidad

Veremos dos técnicas para reducir la dimensionalidad.

Es importante hacer notar que estas técnicas dependen de los modelos: distintos modelos podrían utilizar distintas _features_ de nuestro conjunto de datos.

La primera técnica es más intuitiva, pues simplemente elimina las columnas que sean menos importantes. La información resultante es **interpretable**, dado que simplemente se eliminan columnas enteras.

La segunda técnica es potencialmente más útil, dado que no solo elimina columnas, sino que además fusiona (perdiendo algo de información) la columna que está borrando con otra que **NO** elimina. Es decir, combina la información y después borra la repetición. El resultado **NO** es **interpretable**. Si por ejemplo se fusionan las columnas ```0``` y ```fare``` de nuestro ejemplo, no sabremos exáctamente a qué se referirá.

### Reducción interpretable (selección)

Esta reducción también se llama _selección de características_.

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model = KNeighborsClassifier() # nuestro modelo
sfs = SequentialFeatureSelector(model) # se crea envolviendo al modelo

In [ ]:
sub_X = sfs.fit_transform(X, y) # se usa como el resto
sub_X # mostramos el resultado

array([[ 7.25  ,  0.    ,  0.    ],
       [71.2833,  0.    ,  0.    ],
       [ 7.925 ,  1.    ,  0.    ],
       ...,
       [23.45  ,  0.    ,  0.    ],
       [30.    ,  1.    ,  0.    ],
       [ 7.75  ,  1.    ,  1.    ]])

Podemos explicarlo mostrando las columnas con las que se ha quedado el modelo.

In [ ]:
columns_mask = sfs.get_support() # máscara booleana de las columnas utilizadas
X.columns[columns_mask] # estas son las variables más importantes para este modelo

Index(['fare', 'alone', 'embark_town_0'], dtype='object')

Finalmente, predecimos los resultados.

In [ ]:
model = KNeighborsClassifier()
model = model.fit(sub_X, y) # uso sub_train_X en vez de train_X porque hemos visto que es mejor este subconjunto
pred = model.predict(sub_X) # predigo el resultado final
pred[:10]

array([3, 1, 3, 1, 3, 3, 1, 3, 3, 2])

Ahora lo puedo comprar con lo que ya sé que debe dar.

In [ ]:
sum(pred == y) # casos en los que coincide

844

In [ ]:
sum(pred != y) # casos en los que NO coincide (error del modelo)

47

Pero... ¿y si **NO** hubiese reducido la dimensionalidad?

In [ ]:
model = KNeighborsClassifier()
model = model.fit(X, y) # ahora uso todo el conjunto de datos, no un subconjunto
pred = model.predict(X) # predigo el resultado final
sum(pred == y), sum(pred != y)

(811, 80)

Obtengo un peor resultado.

**Es decir**: fíjate que he mejorado mi modelo **ÚNICAMENTE** realizando una selección de características.

### Reducción **NO** interpretable (extracción)

La segunda técnica nos permitirá _extraer_ la información de algunas características. Recuerda: en estos casos la información que obteníamos no era interpretable.

Como vamos a eliminar toda la interpretabilidad de nuestra información al realizar la extracción, utilizaremos también otra técnica que elimina la información pero mejora enormemente el rendimiento de los modelos. Esta técnica se llama normalización o estandarización (dependiendo de qué métrica se utilice).

Si te acuerdas, al llamar a la función ```describe``` de un ```dataframe```, obteníamos valores para distintos estadísticos (media, cuartiles...). La normalización y la estandarización son procesos de conversión que permiten igualar las escalas en las que tenemos los datos. Para la normalización se usa la media y para la estandarización se usa la media y la desviación típica. Existen otras métricas que permiten escalar los datos, por ejemplo, usando los mínimos y máximos. Un reescalado muy típico es el siguiente:

$$
[\min, \max] → [0, 1]
$$

La mejora que supone este reescalado, aunque a menudo se pierda interpretabilidad, viene dado por trabajar con valores similares. Supongamos que trabajamos con nuestro ejemplo: en una columna tenemos valores entre 0 y 1 (por ejemplo, en las booleanas) y en otra tenemos valores entre 7 y 300 (la tasa de la entrada). Sin quererlo, estamos dando mucha más importancia a la tasa, dado que 300 es mucho más grande que 1. De la otra forma, ese 300 tendrá un valor más cercano a 1, que simbolizará "el valor máximo", de forma que los cálculos serán más sencillos para el modelo.

#### Reescalado de los datos

Aquí solo usaremos la _estandarización_, que resta la media y después divide entre la desviación típica. Importo otros para que te hagas una idea de todo lo que hemos hablado.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

In [ ]:
X.head()

,sex,age,fare,alone,embark_town_0,embark_town_1
0,0,22.0,7.2500,0,0,0
1,1,38.0,71.2833,0,0,1
2,1,26.0,7.9250,1,0,0
3,1,35.0,53.1000,0,0,0
4,0,35.0,8.0500,1,0,0


In [ ]:
ss = StandardScaler() # quizá SS no es el mejor nombre para una variable
ss = ss.fit(X) # primero lo entreno, pero me guardo el escalador para el testeo
sub_X = ss.transform(X)
sub_X[:10]

array([[-0.73769513, -0.5924806 , -0.50244517, -1.2316449 , -0.31191448,
        -0.48557557],
       [ 1.35557354,  0.63878901,  0.78684529, -1.2316449 , -0.31191448,
         2.05941168],
       [ 1.35557354, -0.2846632 , -0.48885426,  0.81192233, -0.31191448,
        -0.48557557],
       [ 1.35557354,  0.40792596,  0.42073024, -1.2316449 , -0.31191448,
        -0.48557557],
       [-0.73769513,  0.40792596, -0.48633742,  0.81192233, -0.31191448,
        -0.48557557],
       [-0.73769513,  0.        , -0.47811643,  0.81192233,  3.20600702,
        -0.48557557],
       [-0.73769513,  1.87005862,  0.39581356,  0.81192233, -0.31191448,
        -0.48557557],
       [-0.73769513, -2.13156761, -0.22408312, -1.2316449 , -0.31191448,
        -0.48557557],
       [ 1.35557354, -0.20770885, -0.42425614, -1.2316449 , -0.31191448,
        -0.48557557],
       [ 1.35557354, -1.20811541, -0.0429555 , -1.2316449 , -0.31191448,
         2.05941168]])

Como ves, hemos perdido toda la interpretabilidad de nuestro conjunto de datos.

#### Técnica del análisis de componentes principales (PCA)

Una de las dos técnicas que vamos a ver se llama PCA. Básicamente elige las componentes (_features_) que mejor expliquen la información que tenemos.

A esta técnica tenemos que darle la cantidad de características con las que nos queremos quedar. Por ejemplo, usaremos 3.

In [ ]:
from sklearn.decomposition import PCA # PCA = Principal Component Analysis

In [ ]:
pca = PCA(n_components=3) # me quedo con 3 columnas
pca = pca.fit(sub_X) # lo entreno
pca_X = pca.transform(sub_X) # predigo mi resultado con el conjunto reducido
pca_X[:10]

array([[-1.65553659e-03, -6.53406152e-01, -1.09161092e+00],
       [-2.51825613e+00,  7.12635787e-01,  6.30890097e-01],
       [ 1.98810483e-01, -6.52554698e-01,  1.03323293e-01],
       [-1.33613891e+00, -6.02684417e-01,  3.74783090e-01],
       [ 1.21866283e+00,  5.44325715e-01, -2.49129247e-01],
       [ 1.83392824e+00, -1.13622798e+00,  2.05052801e+00],
       [ 9.02372414e-01,  1.64526792e+00,  7.29015395e-01],
       [-3.19172091e-01, -1.46975746e+00, -1.88617751e+00],
       [-9.47752683e-01, -1.20063576e+00, -1.29294898e-01],
       [-2.27205358e+00, -5.97287127e-01, -5.49253854e-01]])

Finalmente, entreno mi modelo, a ver qué resultados obtenemos.

In [ ]:
model = KNeighborsClassifier()
model = model.fit(pca_X, y) # fíjate en los nombres de las variables, que los voy cambiando para usar siempre el último conjunto
pred = model.predict(pca_X) # predigo el resultado final
pred[:10] # NO os estoy mintiendo: esto funciona

array([3, 1, 3, 1, 3, 3, 1, 3, 3, 3])

Veamos los resultados.

In [ ]:
sum(pred == y), sum(pred != y)

(751, 140)

En este caso, la extracción ha funcionado peor que la selección.

¿Y si solo usamos los datos estandarizados?

In [ ]:
model = KNeighborsClassifier()
model = model.fit(sub_X, y) # fíjate en los nombres de las variables, que los voy cambiando para usar siempre el último conjunto
pred = model.predict(sub_X) # predigo el resultado final
sum(pred == y), sum(pred != y)

(768, 123)

En efecto, mejora. Esto puede deberse a que hayamos eliminado demasiada información al quedarnos solo con tres _features_. Un análisis completo sería quedarnos secuencialmente con 1, 2, ... y así hasta el máximo posible (6) y ver cómo se comporta el modelo.

Este proceso (el análisis completo) lo veremos en el Tema 4, cuando hablemos de **optimización** de modelos.

#### Técnica de _t-Distributed Stochastic Neighbor Embedding_ (t-SNE para los amigos)

Esta otra técnica (que será la última que veamos), se parece a la técnica de PCA en el sentido de que pierde la interpretabilidad de los datos. La forma de hacerlo, sin embargo, deja de ser un análisis sobre las componentes para convertirse en un estudio estadístico sobre qué variables tienen más importancia (como usa la distibución _t de Stundent_, se llama de esa forma).

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=3) # lo mismo que antes
tsne_X = tsne.fit_transform(sub_X) # TSNE no deja entrenar y después transformar, esto es bastante malo para testear nuestros datos
tsne_X[:10]

array([[ -3.4564705 ,  -0.8768505 , -11.113127  ],
       [ -7.766915  ,   2.418994  ,   0.02302211],
       [ -1.379773  ,  -1.6993227 ,   9.385321  ],
       [ -4.056644  ,  -6.267389  ,   1.9181236 ],
       [  6.3622546 ,   7.9575815 ,   3.596911  ],
       [  8.303201  ,  -6.9989142 ,   5.536873  ],
       [  0.22223163,   7.4348874 ,   9.380019  ],
       [ -6.2095637 ,  -5.0188265 , -10.52854   ],
       [ -6.0502496 , -10.105912  ,   2.4575422 ],
       [-10.477738  ,   2.5621505 ,   0.98327434]], dtype=float32)

In [ ]:
model = KNeighborsClassifier()
model = model.fit(tsne_X, y) # fíjate en los nombres de las variables, que los voy cambiando para usar siempre el último conjunto
pred = model.predict(tsne_X) # predigo el resultado final
sum(pred == y), sum(pred != y)

(763, 128)

Al igual que antes, está a la altura de los datos estandarizados, pero no mejora los resultados obtenidos con éstos. También podríamos analizar la cantidad de componentes que nos queremos quedar con esta técnica, al igual que pasaba con la PCA.

# Resumen

Durante este cuadernillo hemos entrenado nuestro primer modelo de IA. Hemos usado un modelo clásico, como es el KNN, para observar que las dimensiones no siempre son buenas y, en general, preferiremos obtener conjuntos de datos con pocas columnas (o características).

Con los resultados obtenidos (en ```pred```) podríamos ahora mostrar gráficos en los que indicaríamos los resultados para el conjunto de testeo o, por otra parte, podríamos usar distintas métricas para evaluar nuestro modelo (no solo la **precisión** --que es la que hemos usado--).

Todo ello lo dejamos para los temas 3 y 4.